In [119]:
import pandas as pd
import numpy as np

In [120]:
headlines = pd.read_csv('./data/raw/abcnews-date-text.csv')
mm = pd.read_csv('./data/raw/dataverse_files/mmALL_073120_csv.csv')
wgi = pd.read_csv('./data/raw/Data_Extract_From_Worldwide_Governance_Indicators/3f2d17f2-cc70-48f7-8fb9-ab3d88c6b0bd_Data.csv')
pe = pd.read_csv('./data/raw/Data_Extract_From_Poverty_and_Equity/a8619598-0f91-49cc-8c20-8c935bf64574_Data.csv')

In [121]:
pe.head()

,Series Name,Series Code,Country Name,Country Code,1974 [YR1974],1975 [YR1975],1976 [YR1976],1977 [YR1977],1978 [YR1978],1979 [YR1979],...,2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020]
0,Gini index (World Bank estimate),SI.POV.GINI,Afghanistan,AFG,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
1,Gini index (World Bank estimate),SI.POV.GINI,Albania,ALB,..,..,..,..,..,..,...,..,29,..,34.6,32.9,33.7,33.2,..,..,..
2,Gini index (World Bank estimate),SI.POV.GINI,Algeria,DZA,..,..,..,..,..,..,...,27.6,..,..,..,..,..,..,..,..,..
3,Gini index (World Bank estimate),SI.POV.GINI,Angola,AGO,..,..,..,..,..,..,...,..,..,..,..,..,..,..,51.3,..,..
4,Gini index (World Bank estimate),SI.POV.GINI,Argentina,ARG,..,..,..,..,..,..,...,42.6,41.3,40.9,41.6,..,42,41.1,41.3,42.9,..


In [122]:
headlines['publish_date']=pd.to_datetime(headlines['publish_date'], format="%Y%m%d")

In [123]:
headlines.set_index('publish_date', drop=True, inplace=True)

In [124]:
headlines.head()

,headline_text
publish_date,
2003-02-19,aba decides against community broadcasting lic...
2003-02-19,act fire witnesses must be aware of defamation
2003-02-19,a g calls for infrastructure protection summit
2003-02-19,air nz staff in aust strike for pay rise
2003-02-19,air nz strike to affect australian travellers


#### Comparing country names in wgi and mm. 

The following countries are not represented the same in wgi, and I will check for spelling discrepancies:
* 'Venezuela', 'Macedonia', 'Bosnia', 'Yugoslavia',
       'Serbia and Montenegro', 'Russia', 'Cape Verde', 'Gambia',
       'Ivory Coast', 'Congo Brazzaville', 'Congo Kinshasa', 'Swaziland',
       'Iran', 'Egypt', 'Syria', 'Yemen', 'United Arab Emirate',
       'Kyrgyzstan', 'Taiwan', 'North Korea', 'South Korea',
       'Timor Leste'
       * rename wgi 'Venezuela, RB' to 'Venezuela'
       * rename mm Macedonia to North Macedonia
       * rename mm Bosnia to 'Bosnia and Herzegovina'
       * rename mm Yugoslavia to Serbia
       * rename mm Serbia and Montenegro to ??? Serbia? Montenegro? 
       * rename mm Russia to Russian Federation
       * rename mm cape verde to Cabo Verde
       * rename mm Gambia to 'Gambia, The'
       * rename mm Ivory Coast to  'Cote d'Ivoire'
       * rename mm Congo Brazzaville to 'Congo, Rep'
       * rename mm Congo Kinshasa to 'Congo, Dem. Rep.'
       * rename mm Swaziland to 'Eswatini'
       * rename wgi 'Iran, Islamic Rep.' to 'Iran'
       * rename wgi 'Egypt, Arab Rep.' to 'Egypt'
       * rename wgi Syrian Arab Republic to 'Syria'
       * rename wgi 'Yemen, Rep.' to 'Yemen'
       * rename mm 'United Arab Emirate' to 'United Arab Emirates'
       * rename wgi 'Kyrgyz Republic' to 'Kyrgyzstan'
       * rename wgi 'Taiwan, China' to 'Taiwan'
       * rename wgi "Korea, Dem. People's Rep." to 'North Korea'
       * rename wgi 'Korea, Rep.' to 'South Korea'
       * rename mm 'Timor Leste' to 'Timor-Leste'
       
       
       
       
       


In [125]:
mm.head()

,id,country,ccode,year,region,protest,protestnumber,startday,startmonth,startyear,...,protesterdemand4,stateresponse1,stateresponse2,stateresponse3,stateresponse4,stateresponse5,stateresponse6,stateresponse7,sources,notes
0,201990001,Canada,20,1990,North America,1,1,15.0,1.0,1990.0,...,NaN,ignore,NaN,NaN,NaN,NaN,NaN,NaN,1. great canadian train journeys into history;...,canada s railway passenger system was finally ...
1,201990002,Canada,20,1990,North America,1,2,25.0,6.0,1990.0,...,NaN,ignore,NaN,NaN,NaN,NaN,NaN,NaN,1. autonomy s cry revived in quebec the new yo...,protestors were only identified as young peopl...
2,201990003,Canada,20,1990,North America,1,3,1.0,7.0,1990.0,...,NaN,ignore,NaN,NaN,NaN,NaN,NaN,NaN,1. quebec protest after queen calls for unity ...,"the queen, after calling on canadians to remai..."
3,201990004,Canada,20,1990,North America,1,4,12.0,7.0,1990.0,...,NaN,accomodation,NaN,NaN,NaN,NaN,NaN,NaN,1. indians gather as siege intensifies; armed ...,canada s federal government has agreed to acqu...
4,201990005,Canada,20,1990,North America,1,5,14.0,8.0,1990.0,...,NaN,crowd dispersal,arrests,accomodation,NaN,NaN,NaN,NaN,1. dozens hurt in mohawk blockade protest the ...,protests were directed against the state due t...


In [126]:
wgi.head()

,Country Name,Country Code,Series Name,Series Code,1996 [YR1996],1998 [YR1998],2000 [YR2000],2002 [YR2002],2003 [YR2003],2004 [YR2004],...,2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020]
0,Afghanistan,AFG,Control of Corruption: Estimate,CC.EST,-1.291705,-1.180848,-1.29538,-1.263366,-1.351042,-1.345281,...,-1.579174,-1.419741,-1.43651,-1.354829,-1.342216,-1.526172,-1.515626,-1.487624,-1.400733,-1.475405
1,Afghanistan,AFG,Control of Corruption: Number of Sources,CC.NO.SRC,2,2,2,2,3,5,...,9,10,11,11,11,10,10,10,10,9
2,Afghanistan,AFG,Control of Corruption: Percentile Rank,CC.PER.RNK,4.301075,9.793815,5.076142,5.050505,5.050505,5.853659,...,0.9478673,2.369668,1.895735,5.288462,6.25,3.365385,3.846154,4.807693,6.730769,5.288462
3,Afghanistan,AFG,"Control of Corruption: Percentile Rank, Lower ...",CC.PER.RNK.LOWER,0,0,0,0,0,0,...,0,0,0,0.4807692,1.442308,0,0,0,1.923077,0
4,Afghanistan,AFG,"Control of Corruption: Percentile Rank, Upper ...",CC.PER.RNK.UPPER,27.41936,31.4433,29.44162,31.81818,18.18182,14.14634,...,5.687204,11.84834,9.952606,12.5,12.5,9.615385,9.615385,10.09615,11.53846,11.05769


In [127]:
# comparing country names in wgi and mm. 

missing_countries_not_in_mm = list(mm[~mm['country'].isin(wgi['Country Name'])]['country'].unique())
missing_countries_not_in_mm[:5]

['Venezuela', 'Germany West', 'Germany East', 'Czechoslovakia', 'Macedonia']

In [128]:
missing_countries_not_in_wgi = list(wgi[~wgi['Country Name'].isin(mm['country'])]['Country Name'].unique())
missing_countries_not_in_wgi[:5]

['American Samoa', 'Andorra', 'Anguilla', 'Antigua and Barbuda', 'Aruba']

In [129]:
def clean_mass_mob_df(df=mm):
    
    #Drop nulls in startday/month/year and endday/month/year:
    df = df.drop(df[df["startyear"].isnull()].index)
    df.reset_index(inplace=True)
    
    #set startyear/month/day and endday/month/year to int, then string, in preparation for concatenation
    df = df.astype({"startyear": int, "startmonth": int, 'startday': int, "endyear": int, "endmonth": int, 'endday': int})
    df = df.astype({"startyear": str, "startmonth": str, 'startday': str, "endyear": str, "endmonth": str, 'endday': str})
    
    #create start date and end date columns and set as datetime object
    df['start_date'] = df['startyear'] + '-' + df['startmonth'] + '-'+ df['startday']
    df['end_date'] = df['endyear'] + '-' + df['endmonth'] + '-'+ df['endday']
    df['start_date'] = pd.to_datetime(df['start_date'], yearfirst=True)
    df['end_date'] = pd.to_datetime(df['end_date'], yearfirst=True)
    
    #setting df to only include dates between 2002 and 2020.
    df = df[(df['start_date']>'2006') & (df['start_date']<'2021')]
    
    #engineer a column to give duration of protest
    df['duration'] = df['end_date'] - df['start_date']
    
    #fill nulls with 'unknown'
    df.fillna('unknown', inplace=True)
    
    #resetting index
    df.reset_index(drop=True,inplace=True)
    
    #replacing country names in mm to align with wgi country names

    country_changes_mm = {'Macedonia': 'North Macedonia',
                          'Bosnia':'Bosnia and Herzegovina',
                          'Yugoslavia':'Serbia',
                          'Serbia and Montenegro': 'Montenegro',
                          'Russia': 'Russian Federation',
                          'Cape Verde': 'Cabo Verde',
                          'Gambia': 'Gambia, The',
                          'Ivory Coast': 'Cote d\'Ivoire',
                          'Congo Brazzaville': 'Congo, Rep.',
                          'Congo Kinshasa':'Congo, Dem. Rep.',
                          'Swaziland': 'Eswatini',
                          'United Arab Emirate': 'United Arab Emirates',
                          'Timor Leste':'Timor-Leste'}

    
    for name in df['country']:
        for k, v in country_changes_mm.items():
            if name == k:
                mm.replace({'country': country_changes_mm}, inplace=True)
                
    participants_category_map = {
        'unknown': 1,
        '50-99': 2,
        '100-999': 3,
        '2000-4999': 4,
        '1000-1999': 5,
        '5000-10000': 6,
        '>10000': 7
    }
    
    df['participants_category'] = df['participants_category'].map(participants_category_map)
    df['protesterviolence'] = df['protesterviolence'].astype(int)
    
    df = df.rename({'country':'country_name'}, axis=1)
        
    #drop original date columns
    df.drop(columns = ['startday', 'startmonth', 'startyear', 'endday', 'endmonth', 'endyear', 'ccode', 'index', 'location', 'participants'], inplace=True)
    
    return df
    

In [130]:
# participants_category_map = {
#     'unknown': 1,
#     '50-99': 2,
#     '100-999': 3,
#     '2000-4999': 4,
#     '1000-1999': 5,
#        '5000-10000': 6,
#     '>10000': 7
# }

In [131]:
mm.columns

Index(['id', 'country', 'ccode', 'year', 'region', 'protest', 'protestnumber',
       'startday', 'startmonth', 'startyear', 'endday', 'endmonth', 'endyear',
       'protesterviolence', 'location', 'participants_category',
       'participants', 'protesteridentity', 'protesterdemand1',
       'protesterdemand2', 'protesterdemand3', 'protesterdemand4',
       'stateresponse1', 'stateresponse2', 'stateresponse3', 'stateresponse4',
       'stateresponse5', 'stateresponse6', 'stateresponse7', 'sources',
       'notes'],
      dtype='object')

In [132]:
clean_mass_mob_df(mm)

,id,country_name,year,region,protest,protestnumber,protesterviolence,participants_category,protesteridentity,protesterdemand1,...,stateresponse3,stateresponse4,stateresponse5,stateresponse6,stateresponse7,sources,notes,start_date,end_date,duration
0,202006001,Canada,2006,North America,1,1,1,1,mohawk indians,police brutality,...,unknown,unknown,unknown,unknown,unknown,1. world briefing americas: canada: police rai...,the police in southwestern ontario stormed a g...,2006-02-21,2006-08-17,177 days
1,202007001,Canada,2007,North America,1,1,0,1,native groups mohawk indians,"political behavior, process",...,unknown,unknown,unknown,unknown,unknown,1. protests by native groups in canada close r...,canadian travelers faced road closings and rai...,2007-06-29,2007-06-29,0 days
2,202008001,Canada,2008,North America,1,1,1,1,unspecified,police brutality,...,unknown,unknown,unknown,unknown,unknown,1. canada: rioting in montreal the new york ti...,"three police officers were injured, one shot i...",2008-08-10,2008-08-10,0 days
3,202009001,Canada,2009,North America,1,1,0,1,tamil canadians,"political behavior, process",...,unknown,unknown,unknown,unknown,unknown,1. the last stand of sri lanka s tamils in can...,the protesters demanded that the canadian gove...,2009-05-13,2009-05-13,0 days
4,202009002,Canada,2009,North America,1,2,1,1,quebec separatists,"political behavior, process",...,unknown,unknown,unknown,unknown,unknown,1. quebec separatists hurl eggs at prince the ...,protesters hurled eggs and chanted majesty go ...,2009-11-10,2009-11-10,0 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8433,9102014001,Papua New Guinea,2014,Oceania,1,1,1,3,asylum seekers,"political behavior, process",...,unknown,unknown,unknown,unknown,unknown,probe into killing of manus detainee; manus is...,? a government inquiry will be launched as ser...,2014-02-16,2014-02-18,2 days
8434,9102016001,Papua New Guinea,2016,Oceania,1,1,1,5,university students,removal of politician,...,crowd dispersal,unknown,unknown,unknown,unknown,papua new guinea: reports of up to four people...,police in papua new guinea fired gunshots wedn...,2016-05-15,2016-06-09,25 days
8435,9102017001,Papua New Guinea,2017,Oceania,1,1,0,2,protesters opposed to renewing the licence of ...,"political behavior, process",...,unknown,unknown,unknown,unknown,unknown,bougainville imposes moratorium on panguna min...,the bougainville government has enacted an ind...,2017-06-15,2017-06-15,0 days
8436,9102017002,Papua New Guinea,2017,Oceania,1,2,1,2,protesters opposed to counting irregularities ...,"political behavior, process",...,unknown,unknown,unknown,unknown,unknown,"violence, chaos and fraud: fraught papua new g...",peter o neill has been reappointed as prime mi...,2017-07-15,2017-07-15,0 days


In [133]:
mm = clean_mass_mob_df(mm)

In [134]:
mm.head()

,id,country_name,year,region,protest,protestnumber,protesterviolence,participants_category,protesteridentity,protesterdemand1,...,stateresponse3,stateresponse4,stateresponse5,stateresponse6,stateresponse7,sources,notes,start_date,end_date,duration
0,202006001,Canada,2006,North America,1,1,1,1,mohawk indians,police brutality,...,unknown,unknown,unknown,unknown,unknown,1. world briefing americas: canada: police rai...,the police in southwestern ontario stormed a g...,2006-02-21,2006-08-17,177 days
1,202007001,Canada,2007,North America,1,1,0,1,native groups mohawk indians,"political behavior, process",...,unknown,unknown,unknown,unknown,unknown,1. protests by native groups in canada close r...,canadian travelers faced road closings and rai...,2007-06-29,2007-06-29,0 days
2,202008001,Canada,2008,North America,1,1,1,1,unspecified,police brutality,...,unknown,unknown,unknown,unknown,unknown,1. canada: rioting in montreal the new york ti...,"three police officers were injured, one shot i...",2008-08-10,2008-08-10,0 days
3,202009001,Canada,2009,North America,1,1,0,1,tamil canadians,"political behavior, process",...,unknown,unknown,unknown,unknown,unknown,1. the last stand of sri lanka s tamils in can...,the protesters demanded that the canadian gove...,2009-05-13,2009-05-13,0 days
4,202009002,Canada,2009,North America,1,2,1,1,quebec separatists,"political behavior, process",...,unknown,unknown,unknown,unknown,unknown,1. quebec separatists hurl eggs at prince the ...,protesters hurled eggs and chanted majesty go ...,2009-11-10,2009-11-10,0 days


In [135]:
# checking for 60 countries with most appearances in mass mobilization dataset

mm_country_vals = pd.DataFrame(mm['country_name'].value_counts()[:60])['country_name'].index
mm_country_vals

Index(['Kenya', 'United Kingdom', 'Greece', 'Ireland', 'Germany', 'France',
       'China', 'Bangladesh', 'Thailand', 'India', 'Nigeria', 'Venezuela',
       'Yemen', 'Russian Federation', 'Namibia', 'Brazil', 'Kyrgyzstan',
       'Spain', 'Ukraine', 'Pakistan', 'Cyprus', 'Egypt', 'Turkey', 'Cuba',
       'Lebanon', 'Guinea', 'Iraq', 'Malaysia', 'Nepal', 'Italy', 'Chile',
       'Algeria', 'South Korea', 'Iran', 'Romania', 'Mexico', 'Madagascar',
       'Peru', 'Myanmar', 'Haiti', 'Burundi', 'South Africa', 'Sudan',
       'Honduras', 'Taiwan', 'Bolivia', 'Eswatini', 'Armenia', 'Indonesia',
       'Zimbabwe', 'Colombia', 'Hungary', 'Guatemala', 'Togo', 'Burkina Faso',
       'Mauritania', 'Moldova', 'Congo, Dem. Rep.', 'North Macedonia',
       'Ghana'],
      dtype='object')

In [136]:
mm['year'].describe()

count    8438.000000
mean     2013.381489
std         3.940517
min      2006.000000
25%      2010.000000
50%      2014.000000
75%      2017.000000
max      2020.000000
Name: year, dtype: float64

For my initial modeling, I will create 6 models based on their freedom rating in 2006:
1. A country that was classified as 'Free' in 2006 and appeared frequently the mass mobilization dataset.
2. A country that was classified as 'Free' in 2006 and appeared sparsely in the mass mobilization dataset.
3. A country that was classified as 'Not Free' in 2006 and appears frequently in the mass mobilization dataset.
4. A country that was classified as 'Not Free' in 2006 and appears sparsely the mass mobilization dataset.
5. A country that was classified as 'Partially Free' in 2006 and appears frequently the mass mobilization dataset.
6. A country that was classified as 'Partially Free' in 2006 and appears sparsely the mass mobilization dataset.

In order to determine which countries to model, I will cross examine the mass mobilization dataset and the Freedom in the World dataset. 


In [137]:
fiw = pd.read_csv('./data/transformed/fiw_agg.csv')

In [138]:
fiw.head()

,Country/Territory,Region,C/T?,Edition,Status,PR Rating,CL Rating,A,B,C,...,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,x
0,Abkhazia,Eurasia,t,2021,PF,5,5,5,8,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,Asia,c,2021,NF,5,6,3,7,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Albania,Europe,c,2021,PF,3,3,8,12,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Algeria,MENA,c,2021,NF,6,5,3,4,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Andorra,Europe,c,2021,F,1,1,12,15,11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [139]:
fiw.drop(columns=['Country/Territory', 'Status']).columns

Index(['Region', 'C/T?', 'Edition', 'PR Rating', 'CL Rating', 'A', 'B', 'C',
       'Add Q', 'Add A', 'PR', 'D', 'E', 'F', 'G', 'CL', 'Total',
       'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22',
       'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26',
       'Unnamed: 27', 'Unnamed: 28', 'Unnamed: 29', 'Unnamed: 30',
       'Unnamed: 31', 'Unnamed: 32', 'Unnamed: 33', 'Unnamed: 34', 'x'],
      dtype='object')

In [140]:
def clean_fiw_for_model(df=fiw):
    
    columns_to_drop = df.drop(columns=['Country/Territory', 'Status', 'Edition']).columns
    
    df.drop(columns=columns_to_drop, inplace=True)
    
    fiw_column_dict = {}
    for column in df.columns:
        if column == 'Country/Territory':
            fiw_column_dict[column] = 'country'
        elif column == 'C/T?':
            fiw_column_dict[column] = 'country_territory'
        else:
            fiw_column_dict[column] = 'fiw_' + column.lower().replace(' ', '_')

    df.rename(mapper=fiw_column_dict, axis=1, inplace=True)

    country_map = {
        'Slovakia': 'Slovak Republic',
        'Serbia and Montenegro': 'Serbia'   
    }

    for k, v in country_map.items():
        for country in df['country']:
            if country == k:
                country = v
  
    
    df['fiw_edition'] = pd.to_datetime(df['fiw_edition'], format='%Y').dt.year
    
    return df

In [141]:
fiw_clean = clean_fiw_for_model(fiw)

In [142]:
fiw_clean['fiw_edition'] = pd.to_datetime(fiw_clean['fiw_edition'], format='%Y').dt.year

In [143]:
fiw_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3339 entries, 0 to 3338
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   country      3339 non-null   object
 1   fiw_edition  3339 non-null   int64 
 2   fiw_status   3339 non-null   object
dtypes: int64(1), object(2)
memory usage: 78.4+ KB


In [144]:

fiw_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3339 entries, 0 to 3338
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   country      3339 non-null   object
 1   fiw_edition  3339 non-null   int64 
 2   fiw_status   3339 non-null   object
dtypes: int64(1), object(2)
memory usage: 78.4+ KB


In [145]:
fiw_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3339 entries, 0 to 3338
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   country      3339 non-null   object
 1   fiw_edition  3339 non-null   int64 
 2   fiw_status   3339 non-null   object
dtypes: int64(1), object(2)
memory usage: 78.4+ KB


In [146]:
fiw_clean.describe()

,fiw_edition
count,3339.000000
mean,2013.521114
std,4.606546
min,2006.000000
25%,2010.000000
50%,2014.000000
75%,2018.000000
max,2021.000000


In [147]:
#checking for countries categorized as 'Not Free' in 2006

fiw_2006_not_free_list = list(fiw_clean[(fiw_clean['fiw_edition'] == 2006) & (fiw_clean['fiw_status'] == 'NF')]['country'])
fiw_2006_partially_free_list = list(fiw_clean[(fiw_clean['fiw_edition'] == 2006) & (fiw_clean['fiw_status'] == 'PF')]['country'])
fiw_2006_free_list = list(fiw_clean[(fiw_clean['fiw_edition'] == 2006) & (fiw_clean['fiw_status'] == 'F')]['country'])
fiw_2021_not_free_list = list(fiw_clean[(fiw_clean['fiw_edition'] == 2021) & (fiw_clean['fiw_status'] == 'NF')]['country'])
fiw_2021_partially_free_list = list(fiw_clean[(fiw_clean['fiw_edition'] == 2021) & (fiw_clean['fiw_status'] == 'PF')]['country'])
fiw_2021_free_list = list(fiw_clean[(fiw_clean['fiw_edition'] == 2021) & (fiw_clean['fiw_status'] == 'F')]['country'])


In [148]:
#check for which countries in the various Freedom Scales categories by year occur most frequently in the Mass Mobilization data. 
nf_2006_mm_frequent = []
for country in fiw_2006_not_free_list:
    if country in mm_country_vals:
        nf_2006_mm_frequent.append(country)

print(nf_2006_mm_frequent)


['Algeria', 'China', 'Cuba', 'Egypt', 'Guinea', 'Haiti', 'Iran', 'Iraq', 'Myanmar', 'Nepal', 'Pakistan', 'Sudan', 'Togo', 'Zimbabwe']


Algeria, China, Cuba, Egypt, Guinea, Iran, Iraq, Nepal, Pakistan, Russia intersect as appearing frequently in the mass mobilization dataset and were given the category of 'Not Free' in 2006. I will begin by reviewing these options for my first model. 

In [149]:
def get_freedom_status_2006_2021(country_list):
    for country in country_list:
        if country in fiw_2021_free_list:
            print(f'{country} status is "Free" in 2021')
        if country in fiw_2021_partially_free_list:
            print(f'{country} status is "Partially Free" in 2021')
        if country in fiw_2021_not_free_list:
            print(f'{country} status is "Not Free" in 2021')


In [150]:
get_freedom_status_2006_2021(nf_2006_mm_frequent)

Algeria status is "Not Free" in 2021
China status is "Not Free" in 2021
Cuba status is "Not Free" in 2021
Egypt status is "Not Free" in 2021
Guinea status is "Partially Free" in 2021
Haiti status is "Partially Free" in 2021
Iran status is "Not Free" in 2021
Iraq status is "Not Free" in 2021
Myanmar status is "Not Free" in 2021
Nepal status is "Partially Free" in 2021
Pakistan status is "Partially Free" in 2021
Sudan status is "Not Free" in 2021
Togo status is "Partially Free" in 2021
Zimbabwe status is "Not Free" in 2021


Unfortunately, it doesn't appear that the needle has move significantly for many of these countries, thought we do see a shift to 'partially free' for Guinea and Pakistan.

Based on this, I will begin by modeling Pakistan. 

In [151]:
pf_2006_mm_frequent = []
for country in fiw_2006_partially_free_list:
    if country in mm_country_vals:
        pf_2006_mm_frequent.append(country)

print(pf_2006_mm_frequent)

['Armenia', 'Bangladesh', 'Bolivia', 'Burkina Faso', 'Burundi', 'Colombia', 'Guatemala', 'Honduras', 'Kenya', 'Kyrgyzstan', 'Lebanon', 'Madagascar', 'Malaysia', 'Mauritania', 'Moldova', 'Nigeria', 'Thailand', 'Turkey', 'Venezuela', 'Yemen']


In [152]:
get_freedom_status_2006_2021(pf_2006_mm_frequent)

Armenia status is "Partially Free" in 2021
Bangladesh status is "Partially Free" in 2021
Bolivia status is "Partially Free" in 2021
Burkina Faso status is "Partially Free" in 2021
Burundi status is "Not Free" in 2021
Colombia status is "Partially Free" in 2021
Guatemala status is "Partially Free" in 2021
Honduras status is "Partially Free" in 2021
Kenya status is "Partially Free" in 2021
Kyrgyzstan status is "Not Free" in 2021
Lebanon status is "Partially Free" in 2021
Madagascar status is "Partially Free" in 2021
Malaysia status is "Partially Free" in 2021
Mauritania status is "Partially Free" in 2021
Moldova status is "Partially Free" in 2021
Nigeria status is "Partially Free" in 2021
Thailand status is "Not Free" in 2021
Turkey status is "Not Free" in 2021
Venezuela status is "Not Free" in 2021
Yemen status is "Not Free" in 2021


In [153]:
f_2006_mm_frequent = []
for country in fiw_2006_free_list:
    if country in mm_country_vals:
        f_2006_mm_frequent.append(country)

print(f_2006_mm_frequent)

['Brazil', 'Chile', 'Cyprus', 'France', 'Germany', 'Ghana', 'Greece', 'Hungary', 'India', 'Indonesia', 'Ireland', 'Italy', 'Mexico', 'Namibia', 'Peru', 'Romania', 'South Africa', 'South Korea', 'Spain', 'Taiwan', 'Ukraine', 'United Kingdom']


In [62]:
get_freedom_status_2006_2021(f_2006_mm_frequent)

Brazil status is "Free" in 2021
Chile status is "Free" in 2021
Cyprus status is "Free" in 2021
France status is "Free" in 2021
Germany status is "Free" in 2021
Ghana status is "Free" in 2021
Greece status is "Free" in 2021
Hungary status is "Partially Free" in 2021
India status is "Partially Free" in 2021
Indonesia status is "Partially Free" in 2021
Ireland status is "Free" in 2021
Italy status is "Free" in 2021
Mexico status is "Partially Free" in 2021
Namibia status is "Free" in 2021
Peru status is "Partially Free" in 2021
Romania status is "Free" in 2021
South Africa status is "Free" in 2021
South Korea status is "Free" in 2021
Spain status is "Free" in 2021
Taiwan status is "Free" in 2021
Ukraine status is "Partially Free" in 2021
United Kingdom status is "Free" in 2021


In [63]:
# Looking for which countries have seen variation in freedom status from 2006-2021

# I referenced stackoverflow for guidance on re-typing dictionary values

status_dict = {}
for i in fiw['country'].unique():
   status_dict[i] = [
         fiw['fiw_status'][j]
      for j in fiw[fiw['country'] == i].index
   ]

for keys in status_dict:
    status_dict[keys] = set(status_dict[keys])
    

In [64]:
for k, v in status_dict.items():
    if k in nf_2006_mm_frequent:
        print(k, v)

Algeria {'NF'}
China {'NF'}
Cuba {'NF'}
Egypt {'PF', 'NF'}
Guinea {'PF', 'NF'}
Haiti {'PF', 'NF'}
Iran {'NF'}
Iraq {'NF'}
Myanmar {'PF', 'NF'}
Nepal {'PF', 'NF'}
Pakistan {'PF', 'NF'}
Sudan {'NF'}
Togo {'PF', 'NF'}
Zimbabwe {'PF', 'NF'}


In [65]:
for k, v in status_dict.items():
    if k in pf_2006_mm_frequent:
        print(k, v)

Armenia {'PF'}
Bangladesh {'PF'}
Bolivia {'PF'}
Burkina Faso {'PF'}
Burundi {'PF', 'NF'}
Colombia {'PF'}
Guatemala {'PF'}
Honduras {'PF'}
Kenya {'PF'}
Kyrgyzstan {'PF', 'NF'}
Lebanon {'PF'}
Madagascar {'PF'}
Malaysia {'PF'}
Mauritania {'PF', 'NF'}
Moldova {'PF'}
Nigeria {'PF'}
Thailand {'PF', 'NF'}
Turkey {'PF', 'NF'}
Venezuela {'PF', 'NF'}
Yemen {'PF', 'NF'}


In [66]:
for k, v in status_dict.items():
    if k in f_2006_mm_frequent:
        print(k, v)

Brazil {'F'}
Chile {'F'}
Cyprus {'F'}
France {'F'}
Germany {'F'}
Ghana {'F'}
Greece {'F'}
Hungary {'PF', 'F'}
India {'PF', 'F'}
Indonesia {'PF', 'F'}
Ireland {'F'}
Italy {'F'}
Mexico {'PF', 'F'}
Namibia {'F'}
Peru {'PF', 'F'}
Romania {'F'}
South Africa {'F'}
South Korea {'F'}
Spain {'F'}
Taiwan {'F'}
Ukraine {'PF', 'F'}
United Kingdom {'F'}


In [67]:
for k, v in status_dict.items():
    if len(v) == 3:
        print(k, v)

Mali {'PF', 'NF', 'F'}
Tunisia {'PF', 'NF', 'F'}


In [71]:
mm[mm['country_name'] == 'Mali'].shape

(33, 26)

In [72]:
mm[mm['country_name'] == 'Tunisia'].shape

(43, 26)

In [73]:
mm[mm['country_name'] == 'Venezuela'].shape

(138, 26)

In [75]:
mm[mm['country_name'] == 'Kyrgyzstan'].shape

(124, 26)

In [76]:
mm[mm['country_name'] == 'Kazakhstan'].shape

(41, 26)

World Governance Indicators I want to use:
* Control of Corruption: Estimate (CC.EST)
    * Control of Corruption captures perceptions of the extent to which public power is exercised for private gain
* Government Effectiveness: Estimate (GE.EST)
    * Government Effectiveness captures perceptions of the quality of public service
* Political Stability and Absence of Violence/Terrorism: Estimate (PV.EST)
    * Political Stability and Absence of Violence/Terrorism measures perceptions of the likelihood of political instability and/or politically-motivated violence
* Rule of Law: Estimate
    * Rule of Law captures perceptions of the extent to which agents have confidence in and abide by the rules of society
* Voice and Accountability: Estimate
    * Voice and Accountability captures perceptions of the extent to which a country's citizens are able to participate in selecting their government

In [77]:
wgi.head()

,Country Name,Country Code,Series Name,Series Code,1996 [YR1996],1998 [YR1998],2000 [YR2000],2002 [YR2002],2003 [YR2003],2004 [YR2004],...,2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020]
0,Afghanistan,AFG,Control of Corruption: Estimate,CC.EST,-1.291705,-1.180848,-1.29538,-1.263366,-1.351042,-1.345281,...,-1.579174,-1.419741,-1.43651,-1.354829,-1.342216,-1.526172,-1.515626,-1.487624,-1.400733,-1.475405
1,Afghanistan,AFG,Control of Corruption: Number of Sources,CC.NO.SRC,2,2,2,2,3,5,...,9,10,11,11,11,10,10,10,10,9
2,Afghanistan,AFG,Control of Corruption: Percentile Rank,CC.PER.RNK,4.301075,9.793815,5.076142,5.050505,5.050505,5.853659,...,0.9478673,2.369668,1.895735,5.288462,6.25,3.365385,3.846154,4.807693,6.730769,5.288462
3,Afghanistan,AFG,"Control of Corruption: Percentile Rank, Lower ...",CC.PER.RNK.LOWER,0,0,0,0,0,0,...,0,0,0,0.4807692,1.442308,0,0,0,1.923077,0
4,Afghanistan,AFG,"Control of Corruption: Percentile Rank, Upper ...",CC.PER.RNK.UPPER,27.41936,31.4433,29.44162,31.81818,18.18182,14.14634,...,5.687204,11.84834,9.952606,12.5,12.5,9.615385,9.615385,10.09615,11.53846,11.05769


In [78]:
def clean_databank(df, csv_name):
    
    #dropping country code column
    df.drop(columns=['Country Code', 'Series Name'], inplace=True)
    
    #rename year columns
    for column in df.columns[2:]:
        df.rename(columns = {column: column[:4]}, inplace=True)
        
    for column in df.columns[:2]:
        df.rename(columns = {column: column.lower().replace(' ', '_')}, inplace=True)
        
    #create columns for 1997, 1999, and 2001, which will replicate the previous year
    df['1997'] = df['1996']
    df['1999'] = df['1998']
    df['2001'] = df['2000']
    
    #reorder columns
    df = df[['country_name', 'series_code', '1996', '1997', '1998', '1999', '2000',
       '2001','2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010',
       '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019',
       '2020']]
    
    #dropping unneeded series codes
    db_series = ['CC.EST', 'GE.EST', 'PV.NO.SRC', 'RL.EST', 'VA.EST', 'SI.POV.GINI', 'SP.POP.TOTL', 'SI.SPR.BL50.ZS', 'SI.POV.ATTM.MI',
                 'SI.POV.ENRL.MI', 'SI.POV.WATR.MI', 'SI.POV.ELEC.MI', 'SI.DST.05TH.20']
    df = df[df['series_code'].isin(db_series)]
    
    #replacing country names in wgi to align with mm
    country_changes_db = {'Venezuela, RB': 'Venezuela',
                          'Iran, Islamic Rep.': 'Iran',
                          'Egypt, Arab Rep.': 'Egypt',
                          'Syrian Arab Republic': 'Syria',
                          'Yemen, Rep.': 'Yemen',
                          'Kyrgyz Republic': 'Kyrgyzstan',
                          'Taiwan, China': 'Taiwan',
                          'Korea, Dem. People\'s Rep.': 'North Korea',
                          'Korea, Rep.': 'South Korea'}

    for name in df['country_name']:
        for k, v in country_changes_db.items():
            if name == k:
                df.replace({'country_name': country_changes_db}, inplace=True)
    
    df.reset_index(drop=True, inplace=True)
    
    # using melt to get rows and country_name set for merging with other dataframes
    # referenced stackoverflow for melt documentation: https://stackoverflow.com/questions/28654047/convert-columns-into-rows-with-pandas
    df = df.melt(id_vars=['country_name', 'series_code'])
    df.rename(columns = {'variable': 'year', 'value': 'score'}, inplace=True)
    
    # setting year as datetime object
    df['year'] = pd.to_datetime(df['year'], format='%Y').dt.year
    
    df_pivot = df.pivot(index=['country_name','year'], columns=['series_code'], values=['score'])
    df = pd.DataFrame(df_pivot)
    df.replace('..',np.NaN, inplace=True)
    
#     df.to_csv(f'./data/transformed/{csv_name}_pivot.csv')

    return df
    


In [79]:
wgi = clean_databank(wgi, 'wgi')

In [80]:
pe = clean_databank(pe, 'pe')

In [81]:
pe.isna().sum()

       series_code   
score  SI.DST.05TH.20    3440
       SI.POV.ATTM.MI    4801
       SI.POV.ELEC.MI    4804
       SI.POV.ENRL.MI    4843
       SI.POV.GINI       3438
       SI.POV.WATR.MI    4804
       SI.SPR.BL50.ZS    4950
       SP.POP.TOTL        392
dtype: int64

In [82]:
wgi.isna().sum()

       series_code
score  CC.EST         240
       GE.EST         260
       PV.NO.SRC      242
       RL.EST         158
       VA.EST         185
dtype: int64

In [154]:
pe_reset = pd.read_csv('./data/transformed/pe_pivot.csv')

In [155]:
wgi_reset = pd.read_csv('./data/transformed/wgi_pivot.csv')

In [156]:
pe_reset.head()

,country_name,year,SI.DST.05TH.20,SI.POV.ATTM.MI,SI.POV.ELEC.MI,SI.POV.ENRL.MI,SI.POV.GINI,SI.POV.WATR.MI,SI.SPR.BL50.ZS,SP.POP.TOTL
0,Afghanistan,1996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18853444.0
1,Afghanistan,1997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18853444.0
2,Afghanistan,1998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19737770.0
3,Afghanistan,1999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19737770.0
4,Afghanistan,2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20779957.0


In [157]:
wgi_reset.head()

,country_name,year,CC.EST,GE.EST,PV.NO.SRC,RL.EST,VA.EST
0,Afghanistan,1996,-1.291705,-2.175167,2.0,-1.788075,-1.908540
1,Afghanistan,1997,-1.291705,-2.175167,2.0,-1.788075,-1.908540
2,Afghanistan,1998,-1.180848,-2.135942,2.0,-1.737140,-2.039301
3,Afghanistan,1999,-1.180848,-2.135942,2.0,-1.737140,-2.039301
4,Afghanistan,2000,-1.295380,-2.231651,2.0,-1.778437,-2.031417


In [158]:
wgi_reset.shape

(5350, 7)

In [159]:
pe_reset.shape

(4950, 10)

In [160]:
pe_reset['year'].describe()

count    4950.000000
mean     2008.000000
std         7.211831
min      1996.000000
25%      2002.000000
50%      2008.000000
75%      2014.000000
max      2020.000000
Name: year, dtype: float64

In [161]:
wgi_reset['year'].describe()

count    5350.000000
mean     2008.000000
std         7.211777
min      1996.000000
25%      2002.000000
50%      2008.000000
75%      2014.000000
max      2020.000000
Name: year, dtype: float64

In [162]:
wgi_reset['country_name'].value_counts()

Belize                    25
South Sudan               25
Ukraine                   25
Burkina Faso              25
Jamaica                   25
                          ..
Denmark                   25
Honduras                  25
Mozambique                25
Bosnia and Herzegovina    25
Oman                      25
Name: country_name, Length: 214, dtype: int64

In [163]:
pe_reset['country_name'].value_counts()

Belize                                                25
Sub-Saharan Africa (excluding high income)            25
IDA graduates                                         25
Maldives                                              25
Micronesia, Fed. Sts.                                 25
                                                      ..
Poland                                                25
Latin America & Caribbean (IDA total)                 25
Iran                                                  25
Namibia                                               25
Middle East & North Africa (excluding high income)    25
Name: country_name, Length: 198, dtype: int64

In [164]:
pe_countries = pe_reset['country_name'].unique()

In [165]:
mm

,id,country_name,year,region,protest,protestnumber,protesterviolence,participants_category,protesteridentity,protesterdemand1,...,stateresponse3,stateresponse4,stateresponse5,stateresponse6,stateresponse7,sources,notes,start_date,end_date,duration
0,202006001,Canada,2006,North America,1,1,1,1,mohawk indians,police brutality,...,unknown,unknown,unknown,unknown,unknown,1. world briefing americas: canada: police rai...,the police in southwestern ontario stormed a g...,2006-02-21,2006-08-17,177 days
1,202007001,Canada,2007,North America,1,1,0,1,native groups mohawk indians,"political behavior, process",...,unknown,unknown,unknown,unknown,unknown,1. protests by native groups in canada close r...,canadian travelers faced road closings and rai...,2007-06-29,2007-06-29,0 days
2,202008001,Canada,2008,North America,1,1,1,1,unspecified,police brutality,...,unknown,unknown,unknown,unknown,unknown,1. canada: rioting in montreal the new york ti...,"three police officers were injured, one shot i...",2008-08-10,2008-08-10,0 days
3,202009001,Canada,2009,North America,1,1,0,1,tamil canadians,"political behavior, process",...,unknown,unknown,unknown,unknown,unknown,1. the last stand of sri lanka s tamils in can...,the protesters demanded that the canadian gove...,2009-05-13,2009-05-13,0 days
4,202009002,Canada,2009,North America,1,2,1,1,quebec separatists,"political behavior, process",...,unknown,unknown,unknown,unknown,unknown,1. quebec separatists hurl eggs at prince the ...,protesters hurled eggs and chanted majesty go ...,2009-11-10,2009-11-10,0 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8433,9102014001,Papua New Guinea,2014,Oceania,1,1,1,3,asylum seekers,"political behavior, process",...,unknown,unknown,unknown,unknown,unknown,probe into killing of manus detainee; manus is...,? a government inquiry will be launched as ser...,2014-02-16,2014-02-18,2 days
8434,9102016001,Papua New Guinea,2016,Oceania,1,1,1,5,university students,removal of politician,...,crowd dispersal,unknown,unknown,unknown,unknown,papua new guinea: reports of up to four people...,police in papua new guinea fired gunshots wedn...,2016-05-15,2016-06-09,25 days
8435,9102017001,Papua New Guinea,2017,Oceania,1,1,0,2,protesters opposed to renewing the licence of ...,"political behavior, process",...,unknown,unknown,unknown,unknown,unknown,bougainville imposes moratorium on panguna min...,the bougainville government has enacted an ind...,2017-06-15,2017-06-15,0 days
8436,9102017002,Papua New Guinea,2017,Oceania,1,2,1,2,protesters opposed to counting irregularities ...,"political behavior, process",...,unknown,unknown,unknown,unknown,unknown,"violence, chaos and fraud: fraught papua new g...",peter o neill has been reappointed as prime mi...,2017-07-15,2017-07-15,0 days


In [166]:
merged_df = wgi_reset.merge(pe_reset, how='inner', left_on=["country_name", "year"], right_on=["country_name","year"])

In [167]:
mm.head()

,id,country_name,year,region,protest,protestnumber,protesterviolence,participants_category,protesteridentity,protesterdemand1,...,stateresponse3,stateresponse4,stateresponse5,stateresponse6,stateresponse7,sources,notes,start_date,end_date,duration
0,202006001,Canada,2006,North America,1,1,1,1,mohawk indians,police brutality,...,unknown,unknown,unknown,unknown,unknown,1. world briefing americas: canada: police rai...,the police in southwestern ontario stormed a g...,2006-02-21,2006-08-17,177 days
1,202007001,Canada,2007,North America,1,1,0,1,native groups mohawk indians,"political behavior, process",...,unknown,unknown,unknown,unknown,unknown,1. protests by native groups in canada close r...,canadian travelers faced road closings and rai...,2007-06-29,2007-06-29,0 days
2,202008001,Canada,2008,North America,1,1,1,1,unspecified,police brutality,...,unknown,unknown,unknown,unknown,unknown,1. canada: rioting in montreal the new york ti...,"three police officers were injured, one shot i...",2008-08-10,2008-08-10,0 days
3,202009001,Canada,2009,North America,1,1,0,1,tamil canadians,"political behavior, process",...,unknown,unknown,unknown,unknown,unknown,1. the last stand of sri lanka s tamils in can...,the protesters demanded that the canadian gove...,2009-05-13,2009-05-13,0 days
4,202009002,Canada,2009,North America,1,2,1,1,quebec separatists,"political behavior, process",...,unknown,unknown,unknown,unknown,unknown,1. quebec separatists hurl eggs at prince the ...,protesters hurled eggs and chanted majesty go ...,2009-11-10,2009-11-10,0 days


In [168]:
merged_df = mm.merge(merged_df, how='inner', left_on=["country_name", "year"], right_on=["country_name","year"])

In [169]:
merged_df

,id,country_name,year,region,protest,protestnumber,protesterviolence,participants_category,protesteridentity,protesterdemand1,...,RL.EST,VA.EST,SI.DST.05TH.20,SI.POV.ATTM.MI,SI.POV.ELEC.MI,SI.POV.ENRL.MI,SI.POV.GINI,SI.POV.WATR.MI,SI.SPR.BL50.ZS,SP.POP.TOTL
0,202006001,Canada,2006,North America,1,1,1,1,mohawk indians,police brutality,...,1.814535,1.415595,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32571174.0
1,202007001,Canada,2007,North America,1,1,0,1,native groups mohawk indians,"political behavior, process",...,1.814706,1.384672,41.2,NaN,NaN,NaN,33.8,NaN,NaN,32889025.0
2,202008001,Canada,2008,North America,1,1,1,1,unspecified,police brutality,...,1.808290,1.407529,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33247118.0
3,202009001,Canada,2009,North America,1,1,0,1,tamil canadians,"political behavior, process",...,1.804082,1.401846,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33628895.0
4,202009002,Canada,2009,North America,1,2,1,1,quebec separatists,"political behavior, process",...,1.804082,1.401846,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33628895.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8117,9102014001,Papua New Guinea,2014,Oceania,1,1,1,3,asylum seekers,"political behavior, process",...,-0.801859,0.012504,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7946733.0
8118,9102016001,Papua New Guinea,2016,Oceania,1,1,1,5,university students,removal of politician,...,-0.776621,0.158423,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8271766.0
8119,9102017001,Papua New Guinea,2017,Oceania,1,1,0,2,protesters opposed to renewing the licence of ...,"political behavior, process",...,-0.823908,0.132872,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8438038.0
8120,9102017002,Papua New Guinea,2017,Oceania,1,2,1,2,protesters opposed to counting irregularities ...,"political behavior, process",...,-0.823908,0.132872,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8438038.0


In [170]:
merged_df.isna().sum()

id                          0
country_name                0
year                        0
region                      0
protest                     0
protestnumber               0
protesterviolence           0
participants_category       0
protesteridentity           0
protesterdemand1            0
protesterdemand2            0
protesterdemand3            0
protesterdemand4            0
stateresponse1              0
stateresponse2              0
stateresponse3              0
stateresponse4              0
stateresponse5              0
stateresponse6              0
stateresponse7              0
sources                     0
notes                       0
start_date                  0
end_date                    0
duration                    0
CC.EST                      0
GE.EST                      0
PV.NO.SRC                   0
RL.EST                      0
VA.EST                      0
SI.DST.05TH.20           4035
SI.POV.ATTM.MI           7448
SI.POV.ELEC.MI           7458
SI.POV.ENR

In [171]:
mm

,id,country_name,year,region,protest,protestnumber,protesterviolence,participants_category,protesteridentity,protesterdemand1,...,stateresponse3,stateresponse4,stateresponse5,stateresponse6,stateresponse7,sources,notes,start_date,end_date,duration
0,202006001,Canada,2006,North America,1,1,1,1,mohawk indians,police brutality,...,unknown,unknown,unknown,unknown,unknown,1. world briefing americas: canada: police rai...,the police in southwestern ontario stormed a g...,2006-02-21,2006-08-17,177 days
1,202007001,Canada,2007,North America,1,1,0,1,native groups mohawk indians,"political behavior, process",...,unknown,unknown,unknown,unknown,unknown,1. protests by native groups in canada close r...,canadian travelers faced road closings and rai...,2007-06-29,2007-06-29,0 days
2,202008001,Canada,2008,North America,1,1,1,1,unspecified,police brutality,...,unknown,unknown,unknown,unknown,unknown,1. canada: rioting in montreal the new york ti...,"three police officers were injured, one shot i...",2008-08-10,2008-08-10,0 days
3,202009001,Canada,2009,North America,1,1,0,1,tamil canadians,"political behavior, process",...,unknown,unknown,unknown,unknown,unknown,1. the last stand of sri lanka s tamils in can...,the protesters demanded that the canadian gove...,2009-05-13,2009-05-13,0 days
4,202009002,Canada,2009,North America,1,2,1,1,quebec separatists,"political behavior, process",...,unknown,unknown,unknown,unknown,unknown,1. quebec separatists hurl eggs at prince the ...,protesters hurled eggs and chanted majesty go ...,2009-11-10,2009-11-10,0 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8433,9102014001,Papua New Guinea,2014,Oceania,1,1,1,3,asylum seekers,"political behavior, process",...,unknown,unknown,unknown,unknown,unknown,probe into killing of manus detainee; manus is...,? a government inquiry will be launched as ser...,2014-02-16,2014-02-18,2 days
8434,9102016001,Papua New Guinea,2016,Oceania,1,1,1,5,university students,removal of politician,...,crowd dispersal,unknown,unknown,unknown,unknown,papua new guinea: reports of up to four people...,police in papua new guinea fired gunshots wedn...,2016-05-15,2016-06-09,25 days
8435,9102017001,Papua New Guinea,2017,Oceania,1,1,0,2,protesters opposed to renewing the licence of ...,"political behavior, process",...,unknown,unknown,unknown,unknown,unknown,bougainville imposes moratorium on panguna min...,the bougainville government has enacted an ind...,2017-06-15,2017-06-15,0 days
8436,9102017002,Papua New Guinea,2017,Oceania,1,2,1,2,protesters opposed to counting irregularities ...,"political behavior, process",...,unknown,unknown,unknown,unknown,unknown,"violence, chaos and fraud: fraught papua new g...",peter o neill has been reappointed as prime mi...,2017-07-15,2017-07-15,0 days


In [172]:
wgi_countries = wgi_reset['country_name'].unique()

In [173]:
for country in pe_countries:
    if country not in wgi_countries:
        print(country)

East Asia & Pacific
East Asia & Pacific (IDA total)
East Asia & Pacific (excluding high income)
Europe & Central Asia
Europe & Central Asia (IDA total)
Europe & Central Asia (excluding high income)
Fragile and conflict affected situations
High income
IDA countries classified as fragile situations
IDA graduates
IDA total
Latin America & Caribbean
Latin America & Caribbean (IDA total)
Latin America & Caribbean (excluding high income)
Low & middle income
Low income
Lower middle income
Middle East & North Africa
Middle East & North Africa (IDA total)
Middle East & North Africa (excluding high income)
Middle income
South Asia
South Asia (IDA total)
Sub-Saharan Africa
Sub-Saharan Africa (IDA total)
Sub-Saharan Africa (excluding high income)
Upper middle income
World


In [174]:
for country in wgi_countries:
    if country not in pe_countries:
        print(country)

American Samoa
Andorra
Anguilla
Antigua and Barbuda
Aruba
Bahamas, The
Bahrain
Barbados
Bermuda
Brunei Darussalam
Cayman Islands
Cook Islands
Cuba
Dominica
French Guiana
Greenland
Grenada
Guam
Hong Kong SAR, China
Jersey, Channel Islands
Kuwait
Libya
Liechtenstein
Macao SAR, China
Marshall Islands
Martinique
Monaco
Nauru
Netherlands Antilles
New Zealand
Niue
North Korea
Oman
Puerto Rico
Qatar
Reunion
San Marino
Saudi Arabia
Singapore
Somalia
St. Kitts and Nevis
St. Vincent and the Grenadines
Taiwan
Virgin Islands (U.S.)


In [175]:
pe.head()

,Series Name,Series Code,Country Name,Country Code,1974 [YR1974],1975 [YR1975],1976 [YR1976],1977 [YR1977],1978 [YR1978],1979 [YR1979],...,2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020]
0,Gini index (World Bank estimate),SI.POV.GINI,Afghanistan,AFG,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
1,Gini index (World Bank estimate),SI.POV.GINI,Albania,ALB,..,..,..,..,..,..,...,..,29,..,34.6,32.9,33.7,33.2,..,..,..
2,Gini index (World Bank estimate),SI.POV.GINI,Algeria,DZA,..,..,..,..,..,..,...,27.6,..,..,..,..,..,..,..,..,..
3,Gini index (World Bank estimate),SI.POV.GINI,Angola,AGO,..,..,..,..,..,..,...,..,..,..,..,..,..,..,51.3,..,..
4,Gini index (World Bank estimate),SI.POV.GINI,Argentina,ARG,..,..,..,..,..,..,...,42.6,41.3,40.9,41.6,..,42,41.1,41.3,42.9,..


In [176]:
pe[(pe['country_name'] == 'Venezuela') & (pe['score'] == '..')]

KeyError: 'country_name'

In [177]:
wgi.head()

,Country Name,Country Code,Series Name,Series Code,1996 [YR1996],1998 [YR1998],2000 [YR2000],2002 [YR2002],2003 [YR2003],2004 [YR2004],...,2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020]
0,Afghanistan,AFG,Control of Corruption: Estimate,CC.EST,-1.291705,-1.180848,-1.29538,-1.263366,-1.351042,-1.345281,...,-1.579174,-1.419741,-1.43651,-1.354829,-1.342216,-1.526172,-1.515626,-1.487624,-1.400733,-1.475405
1,Afghanistan,AFG,Control of Corruption: Number of Sources,CC.NO.SRC,2,2,2,2,3,5,...,9,10,11,11,11,10,10,10,10,9
2,Afghanistan,AFG,Control of Corruption: Percentile Rank,CC.PER.RNK,4.301075,9.793815,5.076142,5.050505,5.050505,5.853659,...,0.9478673,2.369668,1.895735,5.288462,6.25,3.365385,3.846154,4.807693,6.730769,5.288462
3,Afghanistan,AFG,"Control of Corruption: Percentile Rank, Lower ...",CC.PER.RNK.LOWER,0,0,0,0,0,0,...,0,0,0,0.4807692,1.442308,0,0,0,1.923077,0
4,Afghanistan,AFG,"Control of Corruption: Percentile Rank, Upper ...",CC.PER.RNK.UPPER,27.41936,31.4433,29.44162,31.81818,18.18182,14.14634,...,5.687204,11.84834,9.952606,12.5,12.5,9.615385,9.615385,10.09615,11.53846,11.05769


In [106]:
wgi_pivot = wgi.pivot(index=['country_name','year'], columns=['series_code'], values=['score'])

KeyError: 'country_name'

In [ ]:
wgi_pivot_df = pd.DataFrame(wgi_pivot)

In [ ]:
# wgi_pivot_df.to_csv('./data/transformed/wgi_pivot.csv')

In [ ]:
wgi_scores = pd.read_csv('./data/transformed/wgi_pivot.csv')

In [ ]:
wgi_scores.head(20) 

In [ ]:
wgi.head(20)

In [ ]:
# mm.set_index(['year', 'country'], )

In [ ]:
mm.groupby(['year'])['country'].describe()

In [ ]:
mm['stateresponse1'].value_counts()

In [ ]:
mm['protesterdemand1'].value_counts()

In [ ]:
mm['protesterdemand2'].value_counts()

In [ ]:
# mm.sort_values(by='start_date', inplace=True)

In [ ]:
mm.info()

In [179]:
def prep_for_model(df, country, start_date='2006', end_date='2021'):
    df = df[df['country_name'] == country]
    df.sort_values(by='start_date', inplace=True)
    df.set_index('start_date', inplace=True)
#     df = df[(df.index >= start_date) & (mm.index <= end_date)]
    df = df.loc[start_date:end_date]
    df = pd.concat([df, pd.get_dummies(df['protesterdemand1'])], axis=1)
    df = pd.concat([df, pd.get_dummies(df['stateresponse1'])], axis=1)
    df.drop(columns = ['id', 'country_name', 'region', 'protestnumber', 'year', 'sources', 'end_date', 'protesteridentity','protesterdemand1',
       'protesterdemand2', 'protesterdemand3', 'protesterdemand4',
       'stateresponse1', 'stateresponse2', 'stateresponse3', 'stateresponse4',
       'stateresponse5', 'stateresponse6', 'stateresponse7', 'notes'], inplace=True)
    return df
#     df.to_csv(f'./data/transformed/{country}_model.csv')

In [185]:
kyrgyz_df = prep_for_model(merged_df, 'Kyrgyzstan')
kyrgyz_df.to_csv('./data/transformed/df_for_modeling/kyrgyzstan.csv')

<ipython-input-179-a146f1c2d45e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.sort_values(by='start_date', inplace=True)


In [118]:
mm['protesterviolence'].astype(int)

0       1
1       0
2       1
3       0
4       1
       ..
8433    1
8434    1
8435    0
8436    1
8437    0
Name: protesterviolence, Length: 8438, dtype: int64

In [ ]:
fiw_kgz = fiw_clean[fiw_clean['country'] == 'Kyrgyzstan']

In [ ]:
fiw_kgz.info()

In [ ]:
pd.merge(model_df.assign(grouper=model_df.index),
               fiw_kgz.assign(grouper=fiw_kgz['fiw_edition'].dt.to_period('Y')),
               how='left', on='grouper')

In [ ]:
model_df.shape

In [ ]:
model_df.columns

In [ ]:
model_df.to_csv('./data/transformed/venezuela_model.csv')

In [ ]:
mm['stateresponse2'].value_counts()

In [ ]:
mm['participants_category'].value_counts()

In [ ]:
mm.columns

In [ ]:
mm = pd.concat([mm, pd.get_dummies(mm['protesterdemand1'])], axis=1)

In [ ]:
mm = pd.concat([mm, pd.get_dummies(mm['stateresponse1'])], axis=1)

In [ ]:
mm['protesterdemand1'].unique()

In [ ]:
mm.columns

In [ ]:
mm['police brutality'].sum()

In [ ]:
demand_cols = ['protesterdemand2', 'protesterdemand3', 'protesterdemand4']
response_cols = ['stateresponse2', 'stateresponse3', 'stateresponse4',
       'stateresponse5', 'stateresponse6', 'stateresponse7']
demand_cat = ['price increases, tax policy', 'police brutality',
       'political behavior, process', 'removal of politician',
       'labor wage dispute', 'social restrictions', 'land farm issue']

for cat in demand_cat:
    mm[cat] = [1 for category in column in category ]

In [ ]:
mm['police brutality'].sum()